In [2]:
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker
import sys
import os
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))
from DB.models import init_db, Circuit, Season, RacingWeekend, Driver, Session, SessionResult, Lap, Team, DriverTeamSession, TeamCircuitStats, PitStop
from utils_no_sec import correct_fuel_effect, extract_driver_strategies, get_tyre_deg_per_driver, calculate_base_lap_times

import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from itertools import product

# Initialize database connection
global db_session
engine, db_session = init_db()

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None) 


## 1) Query race into df and pre process

In [3]:
def get_race_session(year, circuit):
	race_session = (db_session.query(Session)
				.join(RacingWeekend, Session.weekend_id == RacingWeekend.racing_weekend_id)
				.join(Circuit, RacingWeekend.circuit_id == Circuit.circuit_id)
				.filter(
					RacingWeekend.year == year,
					Circuit.circuit_name == circuit,
					Session.session_type == "Race"
				)
				.first())
	
	return race_session

def get_race_df(race_session_db):

	laps = race_session_db.laps

	# Convert to DataFrame
	laps_data = []
	for lap in laps:

		laps_data.append({
			"lap_num": lap.lap_num,
			"stint_lap": lap.stint_lap,
			"position": lap.position,
			"driver_name": lap.driver.driver_name,
			"driver_number": lap.driver.driver_num,
			"lap_time": lap.lap_time,
			"tyre_type": lap.tyre_type,
			"tyre_laps": lap.tyre_laps,
			"pit": lap.pit,
			"track_status": lap.track_status
		})


	# Create a DataFrame from the list of dictionaries
	df = pd.DataFrame(laps_data)
	

	df = df.sort_values(["lap_num", "position"]).reset_index(drop=True)

	return df


# def add_race_data(df):
# 	# Get rid of NaN times values
# 	# df = df.dropna(subset=['lap_time']).reset_index(drop=True)

# 	df = df.sort_values(["lap_num", "sector", "position"]).reset_index(drop=True)

# 	# Calculate cumulative race time for each driver
# 	df.loc[:, "cumulative_time"] = df.groupby("driver_name")["lap_time"].cumsum()
 
# 	# Calculate rolling pace (average sector time over the last 5 laps)
# 	df.loc[:, "pace"] = (
# 		df.groupby(["driver_name", "sector"])["lap_time"]
# 		.rolling(window=5, min_periods=1)
# 		.mean()
# 		.reset_index(level=[0, 1], drop=True)
# 	)
	
# 	# Get car ahead"s cumulative time (car immediately ahead in position for each lap)
# 	df.loc[:, "front_cumulative_time"] = df.groupby(["lap_num", "sector"])["cumulative_time"].shift(1)
# 	# This gap is calculated only for drivers who are not in the lead position (position > 1)
# 	df.loc[:, "gap"] = df["cumulative_time"] - df["front_cumulative_time"]
# 	df.loc[:, "gap"] = df["gap"].fillna(0)  # Leader has no car ahead, so gap is 0

# 	# Calculate tyre difference (compared to car immediately ahead in THIS Sector)
# 	df.loc[:, "front_tyre"] = df.groupby(["lap_num", "sector"])["tyre_type"].shift(1)
# 	df.loc[:, "tyre_diff"] = df["front_tyre"] - df["tyre_type"]
# 	df.loc[:, "tyre_diff"] = df["tyre_diff"].fillna(0)  # Leader has no car ahead

# 	# Calculate tyre age difference (compared to car immediately ahead in THIS Sector)
# 	df.loc[:, "front_laps"] = df.groupby(["lap_num", "sector"])["stint_lap"].shift(1)
# 	df.loc[:, "stint_laps_diff"] = df["front_laps"] - df["stint_lap"]
# 	df.loc[:, "stint_laps_diff"] = df["stint_laps_diff"].fillna(0)  # Leader has no car ahead

# 	# Calculate DRS availability (within 1s of car ahead IN THIS Sector)
# 	df.loc[:, "drs_available"] = (
# 		(df["gap"] <= 1) &  # Gap within 1 second
# 		(df["position"] > 1) &  # Not the leader
# 		(df["lap_num"] > 1)  # Not the first lap
# 	)

# 	# Create target variable for overtaking model
# 	df.loc[:, "next_position"] = df.groupby("driver_name")["position"].shift(1)
# 	df.loc[:, "overtaken"] = (
# 		(df["next_position"] < df["position"]) |  # Position improved
# 		(df["next_position"].isna())  # Handle NaN values
# 	)


# 	# Cleanup and final sorting
# 	df = df.drop(columns=["front_cumulative_time", "front_tyre", "next_position"], errors="ignore")

# 	try:
# 		new_order = [
# 			"lap_num", "sector", "stint_num", "stint_lap", "position", "driver_name",
# 			"driver_number", "lap_time", "gap", "cumulative_time", "tyre_type", "tyre_laps", 
# 			"pit", "drs_available", "overtaken", "tyre_diff", "front_laps", "stint_laps_diff", "track_status", "pace"
# 		]

# 		df = df[new_order]
# 	except:
# 		print("Missing column(s)")
# 	return df

def add_race_data(df):
	# Calculate cumulative race time for each driver
	df["cumulative_time"] = df.groupby("driver_name")["lap_time"].cumsum()
 
	# Calculate rolling pace (average lap time over the last 5 laps)
	df["pace"] = (
		df.groupby(["driver_name"])["lap_time"]
		.rolling(window=5, min_periods=1)
		.mean()
		.reset_index(level=[0, 1], drop=True)
	)

	# Get car ahead"s cumulative time (car immediately ahead in position for each lap)
	df["front_cumulative_time"] = df.groupby(["lap_num"])["cumulative_time"].shift(1)
	# This gap is calculated only for drivers who are not in the lead position (position > 1)
	df["gap"] = df["cumulative_time"] - df["front_cumulative_time"]
	df["gap"] = df["gap"].fillna(0)  # Leader has no car ahead, so gap is 0

	# Calculate tyre difference (compared to car immediately ahead in THIS Sector)
	df["front_tyre"] = df.groupby(["lap_num"])["tyre_type"].shift(1)
	df["tyre_diff"] = df["front_tyre"] - df["tyre_type"]
	df["tyre_diff"] = df["tyre_diff"].fillna(0)  # Leader has no car ahead

	# Calculate tyre age difference (compared to car immediately ahead in THIS Sector)
	df["front_laps"] = df.groupby(["lap_num"])["stint_lap"].shift(1)
	df["stint_laps_diff"] = df["front_laps"] - df["stint_lap"]
	df["stint_laps_diff"] = df["stint_laps_diff"].fillna(0)  # Leader has no car ahead

	# Calculate DRS availability (within 1s of car ahead IN THIS Sector)
	df["drs_available"] = (
		(df["gap"] <= 1) &
		(df["position"] > 1) &
		(df["lap_num"] > 1)
	)

	df["next_position"] = df.groupby("driver_name")["position"].shift(1) 
	df["overtaken"] = ((df["next_position"] < df["position"]) | 
					  (df["next_position"].isna()))
	

	# Cleanup and final sorting
	df = df.drop(columns=["front_cumulative_time", "front_tyre", "next_position"])
	df = df.sort_values(["lap_num", "position"]).reset_index(drop=True)

	try:
		new_order = [
			"lap_num", "stint_num", "stint_lap", "position", "driver_name",
			"driver_number", "lap_time", "gap", "cumulative_time", "tyre_type", "tyre_laps", 
			"pit", "drs_available", "overtaken", "tyre_diff", "front_laps", "stint_laps_diff", "track_status", "pace"
		]

		df = df[new_order]
	except:
		print("dont care")
	return df


race_session_db = get_race_session(2022, "Sakhir")

df = get_race_df(race_session_db)
df = add_race_data(df)


# df

dont care


In [4]:
def setup_race_data(race_df, race_session_db):
	driver_strategies = extract_driver_strategies(race_df)
	
	# Extract tyre degradation curves
	driver_tyre_coefficients = get_tyre_deg_per_driver(race_df)
	
	# Precompute driver strategies

	# Correct fuel effects in the race data
	max_laps = race_df["lap_num"].max()
	race_df = correct_fuel_effect(race_df)

	drivers = race_df["driver_number"].unique()

	base_sector_times = calculate_base_lap_times(race_df)

	driver_names = {
		driver: race_df[race_df["driver_number"] == driver]["driver_name"].iloc[0]
		for driver in drivers
	}

	# Precompute fuel corrections
	max_fuel_kg = 110
	fuel_effect_per_kg = 0.03
	fuel_corrections = {
		lap: (max_fuel_kg - (lap - 1) * (max_fuel_kg / max_laps)) * fuel_effect_per_kg
		for lap in range(1, max_laps + 1)
	}


	# 
	session_results = (
		db_session.query(SessionResult.grid_pos, Driver.driver_num, SessionResult.end_status)
		.join(Session, Session.session_id == SessionResult.session_id)
		.join(Driver, Driver.driver_id == SessionResult.driver_id)
		.filter(SessionResult.session_id == race_session_db.session_id)
		.all()
	)
	# Convert session results to a dictionary
	starting_positions = {driver_num: grid_pos for grid_pos, driver_num, _ in session_results}

	# Initialize an empty dictionary to store retirements by lap
	retirements_by_lap = {}

	# Iterate through session results to determine retirements
	for driver_id, driver_num, end_status in session_results:
		# Check if the driver retired (end_status is not "Finished" or "+1 Lap")
		if end_status and not (end_status.startswith("Finished") or end_status.startswith("+")):
			# Find the maximum lap number for the driver (last recorded lap)
			lap_retired = df[df["driver_number"] == driver_num]["lap_num"].max()
			
			# Add the driver to the list of retirees for the corresponding lap
			if lap_retired not in retirements_by_lap:
				retirements_by_lap[lap_retired] = []
			retirements_by_lap[lap_retired].append(driver_num)

	# Get initial positions (starting grid positions) for each driver

	df["starting_position"] = None  # Initialize column with None
	for driver_num, grid_pos in starting_positions.items():
		# Find the first occurrence of the driver
		first_row_index = df[df["driver_number"] == driver_num].index[0]
		df.at[first_row_index, "starting_position"] = grid_pos

		
	
	# Get safety car laps
	filtered_df = race_df[race_df["track_status"] != 1]
	
	# Group by lap_num and convert each group to a list of dictionaries
	slow_laps = {}
	for lap_num, group in filtered_df.groupby("lap_num"):
		slow_laps[lap_num] = group.to_dict(orient="records")
	

	return {
		"driver_tyre_coefficients": driver_tyre_coefficients,
		"driver_strategies": driver_strategies,
		"race_df": race_df,
		"max_laps": max_laps,
		"drivers": drivers,
		"driver_names": driver_names,
		"initial_positions": starting_positions,
		"base_sector_times": base_sector_times,
		"fuel_corrections": fuel_corrections,
		"slow_laps": slow_laps,
		"retirement_laps": retirements_by_lap
	}

### Now create overtaking model

In [5]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.utils.class_weight import compute_sample_weight

# Define features and target
features = [
	"gap",
	"pace",
	"tyre_diff",
	"stint_laps_diff",
	"drs_available",
	"cumulative_time",
	"lap_time",
	"pit"
]

X = df[features]
y = df["overtaken"]

from imblearn.over_sampling import SMOTE



# Train the model on the resampled data

# Handle missing values
X = X.fillna(X.mean())

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)


# Base model
gbc = GradientBoostingClassifier(
	n_estimators=200,
	learning_rate=0.05,
	max_depth=3,
	subsample=0.8,
	random_state=42
)



# Calibrate for better probabilities
model = CalibratedClassifierCV(gbc, method="sigmoid", cv=3)

# Train on all data
model.fit(X_resampled, y_resampled)


feature_means = X.mean()

def predict_overtake(new_race_df):
	# Select the same features used during training
	X_new = new_race_df[features]
	
	# Handle missing values (if any)
	X_new = X_new.fillna(feature_means)
	
	# Make predictions using the trained model
	new_race_df["predicted_overtake"] = model.predict(X_new)
	
	return new_race_df

In [6]:
from sklearn.metrics import accuracy_score, classification_report


# Predict overtakes using the rule-based function
new_race_df = predict_overtake(df)

# Calculate accuracy
actual_overtakes = new_race_df["overtaken"]
predicted_overtakes = new_race_df["predicted_overtake"]
accuracy = accuracy_score(actual_overtakes, predicted_overtakes)
print(f"Accuracy: {accuracy:.3f}")

# Generate classification report
print("\nClassification Report:")
print(classification_report(
	actual_overtakes,
	predicted_overtakes,
	target_names=["No Overtake", "Overtaken"]
))

Accuracy: 0.979

Classification Report:
              precision    recall  f1-score   support

 No Overtake       0.98      1.00      0.99      1002
   Overtaken       1.00      0.80      0.89       123

    accuracy                           0.98      1125
   macro avg       0.99      0.90      0.94      1125
weighted avg       0.98      0.98      0.98      1125



In [7]:
# df = df.dropna(subset=['lap_time']).reset_index(drop=True)
# df = df.sort_values(["lap_num", "sector", "position"]).reset_index(drop=True)
race_data = setup_race_data(df, race_session_db)

(race_data["driver_tyre_coefficients"][44])


{np.int64(1): [0.07278488989689666, -1.2026271852449952, 6.238634096059471],
 np.int64(3): [0.0189699129526459, -0.30788179326652493, 2.9096614995956145],
 np.int64(2): [0.022680988524777766, -0.29443469028850144, 1.6567782783519114]}

In [8]:
def race_sim(precomputed_data, given_driver=None, simulated_strategy=None):
	"""
	simulated_strategy
	"""
	# Extract precomputed data
	driver_tyre_coefficients = precomputed_data["driver_tyre_coefficients"]
	driver_strategies = precomputed_data["driver_strategies"]
	max_laps = precomputed_data["max_laps"]
	drivers = precomputed_data["drivers"]
	driver_names = precomputed_data["driver_names"]
	initial_positions = precomputed_data["initial_positions"]
	base_sector_times = precomputed_data["base_sector_times"]
	fuel_corrections = precomputed_data["fuel_corrections"]
	slow_laps = precomputed_data["slow_laps"]
	retirement_laps = precomputed_data["retirement_laps"]

	drivers_data = []

	if given_driver and simulated_strategy:
		driver_strategies[given_driver] = simulated_strategy

	for driver in drivers:
		drivers_data.append({
			"driver_number": driver,
			"driver_name": driver_names[driver],
			"pit_schedule": driver_strategies[driver],
			"tyre_type": driver_strategies[driver][1],
			"lap_num": 1,
			"sector": 0,
			"lap_time": 0.0,
			"stint_lap": 1,
			"cumulative_time": 0.0,
			"gap": 0.0,
			"pit": False,
			"position": initial_positions[driver],
			"starting_pos": initial_positions[driver],
			"base_sector_times": base_sector_times[driver],
			"pace": 0,  # Initialize pace as 0
			"tyre_diff": 0,  # Initialize tyre difference as 0
			"stint_laps_diff": 0,  # Initialize stint laps difference as 0
			"drs_available": False,  # Initialize DRS availability as False
			"retired": False,
		})
		
	drivers_df = pd.DataFrame(drivers_data)
	
	simulated_data = []
	num_overtakes = 0
	for lap in range(1, max_laps + 1):
		drivers_df["lap_num"] += 1
		drivers_df["stint_lap"] += 1

		if lap in slow_laps:
			slow = True
		else:
			slow = False

		if lap in retirement_laps:
			retiring_drivers = retirement_laps[lap]
			
			# Move all drivers behind the retiring drivers up by 1 position
			for driver in retiring_drivers:
				retiring_position = drivers_df.loc[drivers_df["driver_number"] == driver, "position"].values[0]
				drivers_df.loc[drivers_df["position"] > retiring_position, "position"] -= 1

			# Sort the DataFrame by position to ensure correct ordering
			drivers_df = drivers_df.sort_values(by="position", ascending=True)
			
			# Mark retiring drivers as retired and adjust their position to 999
			drivers_df.loc[drivers_df["driver_number"].isin(retiring_drivers), "retired"] = True
			drivers_df.loc[drivers_df["driver_number"].isin(retiring_drivers), "position"] = 999
	
		
		for index, row in drivers_df.iterrows():
			if row["retired"]:
				continue

			# Handle pit stops at the start of a lap (sector 1)
			if lap in row["pit_schedule"]:
				drivers_df.at[index, "pit"] = True  # Mark pit stop
				drivers_df.at[index, "cumulative_time"] += 20  # Add pit stop penalty
				drivers_df.at[index, "stint_lap"] = 1  # Reset stint lap
				drivers_df.at[index, "tyre_type"] = row["pit_schedule"][lap]  # Change tyre


				# TODO - fix reorder when pit
				# current_time = drivers_df.at[index, "cumulative_time"]
				# for pos in range(row["position"], 21):
				# 	if drivers_df.loc[drivers_df["position"] == pos].iloc[0]["cumulative_time"] > current_time:
				# 		print(pos)


				# drivers_df = drivers_df[~drivers_df["retired"]].sort_values(by="cumulative_time", ascending=True)
				# drivers_df["position"] = range(1, len(drivers_df) + 1)
	
			else:
				drivers_df.at[index, "pit"] = False


			a, b, c = driver_tyre_coefficients[row["driver_number"]][row["tyre_type"]]
			lap_time = (
				row["base_sector_times"]  # Base sector time for specific driver
				+ (a * drivers_df.at[index, "stint_lap"]**2 + b * drivers_df.at[index, "stint_lap"] + c)  # Tyre degradation
				+ fuel_corrections[lap]  # Fuel effect
			)

			# Update sector time and cumulative time
			drivers_df.at[index, "lap_time"] = lap_time
			# if slow:
			# 	lap_time *= 1.5
				
			drivers_df.at[index, "cumulative_time"] += lap_time

			# drivers_df = drivers_df.sort_values(by="cumulative_time", ascending=True)

			# Reassign positions based on the sorted order
			# drivers_df["position"] = range(1, len(drivers_df) + 1)

		# drivers_df = drivers_df.sort_values(by="cumulative_time", ascending=True)
		# drivers_df["position"] = range(1, len(drivers_df) + 1)

		for index, row in drivers_df.iterrows():
			if row["retired"]:
				continue
			ahead_pos = row["position"] - 1

			if ahead_pos > 0:
				ahead_index = drivers_df[drivers_df["position"] == ahead_pos].index[0]

				ahead_pos_time = drivers_df.at[ahead_index, "cumulative_time"]
			
				current_pos_time = row["cumulative_time"]
				if ahead_pos_time > current_pos_time:
					drivers_df.at[ahead_index, "cumulative_time"] = current_pos_time - 0.5

					ahead_pos_time = current_pos_time - 0.5
					
				gap = current_pos_time - ahead_pos_time


				drivers_df.at[index, "gap"] = gap

				
			else:

				gap = 0
				drivers_df.at[index, "gap"] = 0

				

			if gap < 1 and ahead_pos > 0:
				# if ahead_pos > 0:
				ahead_row = drivers_df.loc[drivers_df["position"] == ahead_pos].iloc[0]
					
				drivers_df["pace"] = (
					drivers_df.groupby(["driver_name"])["lap_time"]
					.rolling(window=5, min_periods=1)
					.mean()
					.reset_index(level=[0, 1], drop=True)
				)
				
				# Tyre difference
				drivers_df.at[index, "tyre_diff"] = ahead_row["tyre_type"] - row["tyre_type"]

				# Stint laps difference
				drivers_df.at[index, "stint_laps_diff"] = ahead_row["stint_lap"] - row["stint_lap"]

				# DRS availability
				drivers_df.at[index, "drs_available"] = True

				
				drivers_df = predict_overtake(drivers_df)

				ahead_index = drivers_df[drivers_df["position"] == ahead_pos].index[0]
				# Check if an overtake is predicted
				if drivers_df.at[index, "predicted_overtake"]:
					num_overtakes += 1
					# Swap positions between the current driver and the driver ahead
					current_position = drivers_df.at[index, "position"]

					# Swap positions
					drivers_df.at[index, "position"], drivers_df.at[ahead_index, "position"] = drivers_df.at[ahead_index, "position"], drivers_df.at[index, "position"],

					drivers_df.at[index, "cumulative_time"], drivers_df.at[ahead_index, "cumulative_time"] = drivers_df.at[ahead_index, "cumulative_time"], drivers_df.at[index, "cumulative_time"]
					

	
	drivers_df.drop(columns=["base_sector_times"], inplace=True)
	print(f"Number of overtakes: {num_overtakes}")
	return drivers_df

# print(race_data)
sim_df = race_sim(race_data)


sim_df = sim_df.sort_values(by="position", ascending=True)

# sim_df = sim_df.sort_values(by="cumulative_time", ascending=True)
# sim_df["position"] = range(1, len(sim_df) + 1)

# Reset the index (optional, for cleaner output)
sim_df = sim_df.reset_index(drop=True)
sim_df

Number of overtakes: 34


,driver_number,driver_name,pit_schedule,tyre_type,lap_num,sector,lap_time,stint_lap,cumulative_time,gap,pit,position,starting_pos,pace,tyre_diff,stint_laps_diff,drs_available,retired,predicted_overtake
0,16,Charles Leclerc,"{16: 1, 32: 2, 47: 1, 1: 1}",1,58,0,96.946230,11,5659.099758,0.000000,False,1,1,99.853310,0,0,False,False,False
1,55,Carlos Sainz,"{15: 1, 34: 2, 45: 1, 1: 1}",1,58,0,97.611455,13,5683.276634,24.176876,False,2,3,96.946230,1,-17,True,False,False
2,44,Lewis Hamilton,"{12: 3, 28: 2, 45: 1, 1: 1}",1,58,0,99.094531,13,5686.695835,3.419200,False,3,5,100.246786,0,1,True,False,False
3,20,Kevin Magnussen,"{15: 1, 35: 2, 47: 1, 1: 1}",1,58,0,98.431967,11,5725.138692,38.442857,False,4,7,98.866575,1,-13,True,False,False
4,14,Fernando Alonso,"{12: 2, 26: 3, 43: 1, 1: 1}",1,58,0,100.179132,15,5732.050132,6.911440,False,5,8,98.431967,0,-4,True,False,False
5,77,Valtteri Bottas,"{15: 2, 37: 2, 46: 1, 1: 1}",1,58,0,98.612239,12,5745.763646,14.576411,False,6,6,102.216858,-1,-6,True,False,False
6,63,George Russell,"{16: 3, 34: 2, 46: 1, 1: 1}",1,58,0,97.749342,12,5746.263646,0.500000,False,7,9,97.749342,0,0,True,False,False
7,4,Lando Norris,"{25: 3, 42: 1, 46: 1, 1: 2}",1,58,0,97.593667,12,5758.345141,12.081495,False,8,13,101.033263,0,-8,True,False,False
8,27,Nico Hulkenberg,"{20: 1, 38: 2, 45: 1, 1: 1}",1,58,0,101.033263,13,5773.305115,14.959974,False,9,17,100.675560,0,-1,True,False,False
9,47,Mick Schumacher,"{13: 2, 36: 1, 1: 1}",1,58,0,102.216858,22,5788.146984,14.841869,False,10,12,97.198867,0,-9,True,False,False


In [9]:
import pandas as pd
import numpy as np

def compare_simulation_with_actual(sim_df, race_df):
	# Initialize a list to store comparison results
	comparison_results = []
	
	# Get unique drivers from the simulated DataFrame
	drivers = sim_df["driver_number"].unique()
	
	# Calculate cumulative times for each driver in both simulated and actual data
	for driver in drivers:
		# Simulated cumulative time for the driver
		sim_cumulative_time = sim_df[sim_df["driver_number"] == driver]["cumulative_time"].max()
		
		# Actual cumulative time for the driver
		actual_cumulative_time = race_df[race_df["driver_number"] == driver]["lap_time"].sum()

		# if actual_cumulative_time < 5000:
		# 	continue
		# Calculate the absolute error for the driver
		absolute_error = abs(sim_cumulative_time - actual_cumulative_time)
		
		# Store the results for the driver
		comparison_results.append({
			"driver_number": driver,
			"simulated_cumulative_time": sim_cumulative_time,
			"actual_cumulative_time": actual_cumulative_time,
			"absolute_error": absolute_error
		})
	
	# Convert the results to a DataFrame
	comparison_df = pd.DataFrame(comparison_results)
	
	# Calculate the total Mean Absolute Error (MAE)
	total_mae = comparison_df["absolute_error"].mean()
	
	return comparison_df, total_mae


# Example usage
# Assuming `sim_df` is the output of your simulation and `df` is the actual race DataFrame
comparison_df, total_mae = compare_simulation_with_actual(sim_df, df)



# Print the total MAE
print("\Mean Absolute Error (MAE):", total_mae)
comparison_df

\Mean Absolute Error (MAE): 105.71405240163809


,driver_number,simulated_cumulative_time,actual_cumulative_time,absolute_error
0,16,5659.099758,5538.374,120.725758
1,55,5683.276634,5859.182,175.905366
2,44,5686.695835,5863.259,176.563165
3,20,5725.138692,5868.338,143.199308
4,14,5732.050132,5875.974,143.923868
5,77,5745.763646,5869.703,123.939354
6,63,5746.263646,5714.246,32.017646
7,4,5758.345141,5909.919,151.573859
8,27,5773.305115,5917.413,144.107885
9,47,5788.146984,5886.158,98.011016


In [10]:
def get_accuracy(year, circuit, sim_df):
	# Fetch the actual race session from the database
	race_session = (
		db_session.query(Session)
		.join(RacingWeekend, Session.weekend_id == RacingWeekend.racing_weekend_id)
		.join(Circuit, RacingWeekend.circuit_id == Circuit.circuit_id)
		.filter(
			RacingWeekend.year == year,
			Circuit.circuit_name == circuit,
			Session.session_type == "Race"
		)
		.first()
	)

	if not race_session:
		raise ValueError(f"No race session found for {year} at {circuit}")

	# Fetch the actual race laps and results
	laps = race_session.laps
	session_results = (
		db_session.query(SessionResult.position, Driver.driver_num)
		.join(Session, Session.session_id == SessionResult.session_id)
		.join(Driver, Driver.driver_id == SessionResult.driver_id)
		.filter(SessionResult.session_id == race_session.session_id)
		.all()
	)

	# Convert session_results to a dictionary for easy lookup
	actual_results = {driver_num: position for position, driver_num in session_results}

	# Extract simulated results from sim_df
	sim_df = sim_df[~sim_df["retired"]]
	sim_results = sim_df.groupby("driver_number").last()["position"].to_dict()

	# Ensure both results have the same drivers
	common_drivers = set(actual_results.keys()).intersection(sim_results.keys())
	if not common_drivers:
		raise ValueError("No common drivers found between actual and simulated results")

	# Filter results to only include common drivers
	actual_positions = [actual_results[driver] for driver in common_drivers]
	sim_positions = [sim_results[driver] for driver in common_drivers]

	# Calculate accuracy metrics
	position_accuracy = sum(1 for a, s in zip(actual_positions, sim_positions) if a == s) / len(common_drivers)
	top_3_accuracy = sum(1 for a, s in zip(actual_positions, sim_positions) if (a <= 3 and s <= 3)) / 3
	mean_error = sum(abs(a - s) for a, s in zip(actual_positions, sim_positions)) / len(common_drivers)
	total_error = sum(abs(a - s) for a, s in zip(actual_positions, sim_positions))

	# Return accuracy metrics
	return {
		"position_accuracy": position_accuracy,
		"top_3_accuracy": top_3_accuracy,
		"mean_error": mean_error,
		"total_error": total_error,
	}

get_accuracy(2022, "Sakhir", sim_df)

{'position_accuracy': 0.23529411764705882,
 'top_3_accuracy': 1.0,
 'mean_error': 2.588235294117647,
 'total_error': 44}

{'position_accuracy': 0.35294117647058826,
 'top_3_accuracy': 1.0,
 'mean_error': 2.823529411764706,
 'total_error': 48}

In [11]:
from skopt import gp_minimize
from skopt.space import Integer, Categorical

def optimize_strategy_up_to_3_stops(precomputed_data, driver_number=14, n_calls=100):

	max_laps = precomputed_data["max_laps"]

	# Define the search space
	space = [
		Categorical([1, 2, 3], name='start_tyre'),       # Starting tyre (1=Hard, 2=Medium, 3=Soft)
		Integer(2, max_laps - 1, name='pit1_lap'),      # First pit stop lap (optional)
		Categorical([1, 2, 3], name='pit1_tyre'),        # First pit stop tyre (optional)
		Integer(2, max_laps - 1, name='pit2_lap'),      # Second pit stop lap (optional)
		Categorical([1, 2, 3], name='pit2_tyre'),        # Second pit stop tyre (optional)
		Integer(2, max_laps - 1, name='pit3_lap'),      # Third pit stop lap (optional)
		Categorical([1, 2, 3], name='pit3_tyre'),        # Third pit stop tyre (optional)
	]

	def objective(params):
		start_tyre, pit1_lap, pit1_tyre, pit2_lap, pit2_tyre, pit3_lap, pit3_tyre = params

		# Construct the strategy dictionary
		strategy = {1: start_tyre}  # Always include the starting tyre

		# Add pit stops only if their laps are valid and in ascending order
		pit_laps = sorted(set([pit1_lap, pit2_lap, pit3_lap]))
		tyres = [pit1_tyre, pit2_tyre, pit3_tyre]

		# Ensure pit laps are within bounds and in ascending order
		valid_pit_laps = []
		valid_tyres = []
		for lap, tyre in zip(pit_laps, tyres):
			if 2 <= lap < max_laps:  # Only include valid pit laps
				valid_pit_laps.append(lap)
				valid_tyres.append(tyre)

		# Add valid pit stops to the strategy
		for lap, tyre in zip(valid_pit_laps, valid_tyres):
			strategy[lap] = tyre

		# Penalize strategies with fewer than 2 distinct tyre types
		tyre_types_used = set(strategy.values())
		if len(tyre_types_used) < 2:
			return 20.0  # Penalize with worst position

		# Run the simulation
		try:
			drivers_df = race_sim(precomputed_data, given_driver=driver_number, simulated_strategy=strategy)
			final_position = drivers_df[drivers_df['driver_number'] == driver_number]['position'].iloc[-1]
			return final_position  # Minimize finishing position
		except Exception as e:
			print(f"Error during simulation: {e}")
			return 20.0  # Return worst position in case of errors

	# Perform Bayesian optimization
	result = gp_minimize(
		objective,
		space,
		n_calls=n_calls,
		random_state=42,
		verbose=True
	)

	# Extract best parameters and construct the optimal strategy
	best_params = result.x
	best_strategy = {1: best_params[0]}  # Always include the starting tyre

	# Add valid pit stops to the strategy
	pit_laps = sorted(set([best_params[1], best_params[3], best_params[5]]))
	tyres = [best_params[2], best_params[4], best_params[6]]

	for lap, tyre in zip(pit_laps, tyres):
		if 2 <= lap < precomputed_data["max_laps"]:
			best_strategy[lap] = tyre

	best_position = result.fun

	print(f"Best Strategy: {best_strategy}")
	print(f"Best Finishing Position: {best_position}")

	return best_strategy, best_position

# Example usage
# best_strategy, best_position = optimize_strategy_up_to_3_stops(race_data, driver_number=10, n_calls=100)
# print(f"Best Strategy: {best_strategy}")
# print(f"Best Finishing Position: {best_position}")

In [12]:
from bayes_opt import BayesianOptimization
import numpy as np

def bayesian_strategy_optimization(race_data, given_driver, max_iterations=50):
	"""
	Optimize the pit strategy for a given driver using Bayesian Optimization.
	
	Args:
		race_data (dict): Precomputed race data.
		given_driver (int): The driver number to optimize the strategy for.
		max_iterations (int): Maximum number of iterations for optimization.
		
	Returns:
		tuple: Best strategy (dict) and best finishing position (int).
	"""
	# Extract precomputed data
	max_laps = race_data["max_laps"]
	initial_strategy = race_data["driver_strategies"][given_driver]  # Use the driver's actual race strategy

	# Define the objective function for Bayesian Optimization
	def objective_function(starting_tyre, pit1_lap, pit1_tyre, pit2_lap, pit2_tyre, pit3_lap, pit3_tyre):
		# Ensure pit laps are unique and within valid range
		pit_laps = sorted(set([int(pit1_lap), int(pit2_lap), int(pit3_lap)]))
		tyres = [int(pit1_tyre), int(pit2_tyre), int(pit3_tyre)]
		
		# Filter out invalid pit laps (e.g., out of bounds or overlapping)
		valid_pit_laps = []
		valid_tyres = []
		for lap, tyre in zip(pit_laps, tyres):
			if 2 <= lap < max_laps:  # Only include valid pit laps
				valid_pit_laps.append(lap)
				valid_tyres.append(tyre)

		# Construct the strategy dictionary
		strategy = {1: int(starting_tyre)}  # Starting tyre
		for lap, tyre in zip(valid_pit_laps, valid_tyres):
			strategy[lap] = tyre

		# Penalize strategies with fewer than 2 distinct tyre types
		tyre_types_used = set(strategy.values())
		if len(tyre_types_used) < 2:
			return -20.0  # Penalize with worst position (negative because we're maximizing)

		# Evaluate the strategy using the race simulation
		try:
			sim_df = race_sim(race_data, given_driver=given_driver, simulated_strategy=strategy)
			final_position = sim_df[sim_df["driver_number"] == given_driver]["position"].iloc[-1]
			return -final_position  # Negative because BayesianOptimization maximizes by default
		except Exception as e:
			print(f"Error during simulation: {e}")
			return -20.0  # Return worst position in case of errors

	# Set up the parameter bounds for Bayesian Optimization
	pbounds = {
		"starting_tyre": (1, 3),  # Starting tyre (1=Hard, 2=Medium, 3=Soft)
		"pit1_lap": (2, max_laps - 1),  # First pit stop lap (optional)
		"pit1_tyre": (1, 3),  # First pit stop tyre (optional)
		"pit2_lap": (2, max_laps - 1),  # Second pit stop lap (optional)
		"pit2_tyre": (1, 3),  # Second pit stop tyre (optional)
		"pit3_lap": (2, max_laps - 1),  # Third pit stop lap (optional)
		"pit3_tyre": (1, 3),  # Third pit stop tyre (optional)
	}

	# Initialize the Bayesian Optimizer
	optimizer = BayesianOptimization(
		f=objective_function,
		pbounds=pbounds,
		verbose=2,
		random_state=42
	)

	# Perform the optimization
	optimizer.maximize(init_points=5, n_iter=max_iterations)

	# Extract the best strategy from the optimizer
	best_params = optimizer.max["params"]
	best_starting_tyre = int(best_params["starting_tyre"])
	pit_laps = sorted(set([int(best_params["pit1_lap"]), int(best_params["pit2_lap"]), int(best_params["pit3_lap"])]))
	tyres = [int(best_params["pit1_tyre"]), int(best_params["pit2_tyre"]), int(best_params["pit3_tyre"])]

	# Construct the best strategy dictionary
	best_strategy = {1: best_starting_tyre}
	for lap, tyre in zip(pit_laps, tyres):
		if 2 <= lap < max_laps:  # Only include valid pit laps
			best_strategy[lap] = tyre

	# Evaluate the best strategy to get the finishing position
	sim_df = race_sim(race_data, given_driver=given_driver, simulated_strategy=best_strategy)
	best_position = sim_df[sim_df["driver_number"] == given_driver]["position"].iloc[-1]

	print(f"Best Strategy: {best_strategy}")
	print(f"Best Finishing Position: {best_position}")

	return best_strategy, best_position

# Example Usage
# best_strategy, best_position = bayesian_strategy_optimization(
# 	race_data=race_data,
# 	given_driver=10,
# 	max_iterations=20
# )
